### Initial Setup

In [1]:
!pip install pymupdf faiss-cpu sentence-transformers openai>=1.0.0

### Import Libraries

In [2]:
from google.colab import drive
import os
import fitz
import faiss
import numpy as np
import json
from sentence_transformers import SentenceTransformer
from openai import OpenAI

### Save OpenAI Key

In [3]:
client = OpenAI(api_key="")



### Mount Google Drive where all pdf are saved

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### create Function to extract text from pdf

In [63]:
'''
def estrai_testo_pdf_con_chunking(cartella, chunk_size=200):
    chunks = []
    metadati = []
    for nome_file in os.listdir(cartella):
        if nome_file.endswith(".pdf"):
            path = os.path.join(cartella, nome_file)
            doc = fitz.open(path)
            testo = ""
            for pagina in doc:
                testo += pagina.get_text()
            parole = testo.split()
            for i in range(0, len(parole), chunk_size):
                chunk = " ".join(parole[i:i + chunk_size])
                chunks.append(chunk)
                metadati.append({
                    "file": nome_file,
                    "inizio": i,
                    "fine": i + chunk_size
                })
    return chunks, metadati
'''
def estrai_testo_pdf_con_chunking(cartella, chunk_size=200):
    chunks = []
    metadati = []
    for nome_file in os.listdir(cartella):
        if nome_file.endswith(".pdf"):
            path = os.path.join(cartella, nome_file)
            doc = fitz.open(path)
            testo = ""
            for pagina in doc:
                testo += pagina.get_text()
            parole = testo.split()
            for i in range(0, len(parole), chunk_size):
                chunk = " ".join(parole[i:i + chunk_size])
                # Inseriamo il nome del file come prefisso nel testo stesso
                chunk_con_titolo = f"[{nome_file}]\n{chunk}"
                chunks.append(chunk_con_titolo)
                metadati.append({
                    "file": nome_file,
                    "inizio": i,
                    "fine": i + chunk_size
                })
    return chunks, metadati


### Execute function to extract text

In [71]:
CARTELLA_DISCIPLINARI = '/content/drive/MyDrive/DOC_DOCG'

chunks, metadati = estrai_testo_pdf_con_chunking(CARTELLA_DISCIPLINARI)
print(f"🧩 Totale chunk estratti: {len(chunks)}")

path = "/content/drive/MyDrive/DOC_DOCG/Dati/chunks_chunked.json"

# Salvataggio in JSON
with open(path, "w", encoding="utf-8") as f:
    json.dump(chunks, f, ensure_ascii=False, indent=2)

print(f"✅ File salvato: {path}")

🧩 Totale chunk estratti: 9959
✅ File salvato: /content/drive/MyDrive/DOC_DOCG/Dati/chunks_chunked.json


### Embedding Generation and FAISS Index creation

In [65]:
model = SentenceTransformer('all-MiniLM-L6-v2')

embedding = model.encode(chunks, show_progress_bar=True)

index = faiss.IndexFlatL2(embedding[0].shape[0])
index.add(np.array(embedding))

Batches:   0%|          | 0/312 [00:00<?, ?it/s]

### Index and Metadata Saving

In [66]:


faiss.write_index(index, "/content/drive/MyDrive/DOC_DOCG/Dati/index_chunked.faiss")
with open("/content/drive/MyDrive/DOC_DOCG/Dati/metadata_chunked.json", "w") as f:
    json.dump(metadati, f)

### Upload Faiss Index

In [ ]:
# Carica FAISS index
index = faiss.read_index("/content/drive/MyDrive/DOC_DOCG/Dati/index_chunked.faiss")

# Carica i metadati e i chunk
with open("/content/drive/MyDrive/DOC_DOCG/Dati/metadata_chunked.json", "r") as f:
    metadati = json.load(f)

### Searching and Answer Function with RAG

In [67]:
def query_rag_chunk(domanda, top_k=3):
    emb_domanda = model.encode([domanda])
    distanze, indici = index.search(np.array(emb_domanda), top_k)
    contesti = [chunks[i] for i in indici[0]]
    fonti = [metadati[i]['file'] for i in indici[0]]
    return "\n\n".join(contesti), fonti


### Question Example

In [68]:
domanda_vitigni = "Quali sono i vitigni ammessi nella DOCG Chianti? puoi rispondere solo con un elenco dei vitigni divisi per tipologia? puoi indicare la percentuale massima e se presente la minima? puoi scriverlo come fosse un file JSON?"
domanda_tipologie = "Quali tipologie sono ammesse nella DOC Trento? puoi rispondere solo con un elenco delle tipologie? puoi scriverlo come fosse un file JSON?"
domanda_generica = "quali DOC hanno nel loro disciplinare le MGA? puoi r4aggrupparle in un'elenco?"

domanda = domanda_vitigni

contesto, fonti = query_rag_chunk(domanda, top_k=3)

MAX_TOKEN_ESTIMATO = 3000  # in media 1 token ≈ 4 caratteri
if len(contesto) > MAX_TOKEN_ESTIMATO * 4:
    contesto = contesto[:MAX_TOKEN_ESTIMATO * 4]

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "Sei un esperto di disciplinari del vino italiano."},
        {"role": "user", "content": f"Domanda: {domanda}\n\nContesto:\n{contesto}"}
    ]
)

print(response.choices[0].message.content)

{
  "vini_bianchi": [],
  "vini_rossi": [
    { 
      "vitigno": "Sangiovese",
      "percentuale_massima": 100,
      "percentuale_minima": 70
    },
    {
      "vitigno": "Canaiolo nero",
      "percentuale_massima": 30,
      "percentuale_minima": 0
    },
    {
      "vitigno": "Trebbiano Toscano e Malvasia del Chianti",
      "percentuale_massima": 10,
      "percentuale_minima": 0
    },
    {
      "vitigno": "Colorino, Cabernet Sauvignon, Merlot",
      "percentuale_massima": 20,
      "percentuale_minima": 0
    }
  ],
  "vini_rosati": [],
  "vin_spassiti": [],
  "vini_muffati": [],
  "vini_frizzanti": [],
  "vini_spumanti": [],
  "altro": []
}
